# Notebook 1: Basic Example

Here we will introduce a basic example of how **BATTLESIMULATOR** works and can be of benefit to you.

### Requirements:

- `numpy`
- `pandas`
- `matplotlib`
- `numba`

In [1]:
import sys
# map path back to the directory above battlesim/
sys.path.insert(0,"../")
# our main import
import battlesim as bsm

### Other Imports

In [2]:
from IPython.display import HTML
%matplotlib inline

## Create a Battle using the CloneWars Dataset

In [3]:
battle = bsm.Battle("../datasets/starwars-clonewars.csv")
battle

bsm.Battle(init=False)

### Within the `db_` object, we can see the table of information

This table represents the *knowledge* or statistics of each unit. These can be your own values/unit names, but the following column names **MUST** be present and the same within your own file, ideally in `.csv` file format.

In [4]:
battle.db_

,Allegiance,Type,Armor,HP,Damage,Range,Movement Speed,Accuracy,Miss,allegiance_int
Name,,,,,,,,,,
local militia,Republic,Standard,15,20,10,2.0,0.6,5,30,0
b1 battledroid,CIS,Standard,40,20,15,2.5,0.5,30,35,1
clone trooper,Republic,Standard,45,35,16,3.0,0.7,65,65,0
b2 battledroid,CIS,Standard,100,40,20,3.5,0.4,70,40,1
arc trooper,Republic,Elite,100,100,20,5.0,1.0,90,95,0
bx-series droid commando,CIS,Elite,70,50,22,7.0,0.8,70,95,1
clone sharpshooter,Republic,Specialist,50,40,50,15.0,0.2,60,40,0
battle droid assassin,CIS,Specialist,60,20,50,15.0,0.2,55,45,1
clone commando,Republic,Elite,100,120,20,3.0,0.9,97,97,0


## Initialising Army groups

The other elements within `battle` are not initialized yet, we need to *declare groups of units* using the names on the columns above, followed by the number of that unit we wish to use.

To do this, we expose another object called `Composite`, which holds this meta information for a selection of a *unit group*. A list of these Composites can then be passed when we want to simulate.

The main parameters are:

1. The name of the unit type (from .db_)
2. The number of units within the group
3. (Optional) position of the group using a sampling technique. By default this creates a gaussian distribution at (0, 0).

In [5]:
b1_10s = bsm.Composite("b1 battledroid", 10)
b1_10s

Composite('b1 battledroid', n=10, pos=Sampling('normal', ()), init_ai='nearest', rolling_ai='nearest', decision_ai='aggressive')

We can define the position ourselves using a sampling technique (normal distribution with mean 0, std 2):

In [7]:
comp = [
    bsm.Composite("b1 battledroid", 10, bsm.Sampling("normal")),
    bsm.Composite("clone trooper", 5, bsm.Sampling("normal", 2, 1))
]
comp

[Composite('b1 battledroid', n=10, pos=Sampling('normal', ()), init_ai='nearest', rolling_ai='nearest', decision_ai='aggressive'),
 Composite('clone trooper', n=5, pos=Sampling('normal', (2, 1)), init_ai='nearest', rolling_ai='nearest', decision_ai='aggressive')]

### We can view the armies we have stored:

In [8]:
import numpy as np

In [9]:
battle.create_army(comp)

bsm.Battle(init=False)

### We can view the composition:

You notice that we have some interesting columns that detail what each 'army' is doing:

1. *position*: this defines how to spawn the unit, i.e with a normal distribution
2. *init_ai*: this defines how the army will find the nearest enemy at the start
3. *rolling_ai*: this defines how the army will find the nearest enemy throughout simulation progression
4. *decision_ai*: a broad suite of AI algorithms that define how the 'army' behaves. by default they act aggressively by chasing the enemy.

In [10]:
battle.composition_

[Composite('b1 battledroid', n=10, pos=Sampling('normal', ()), init_ai='nearest', rolling_ai='nearest', decision_ai='aggressive'),
 Composite('clone trooper', n=5, pos=Sampling('normal', (2, 1)), init_ai='nearest', rolling_ai='nearest', decision_ai='aggressive')]

### Internally, a matrix `M` is created:

Here the *M* stands for mutability.

This enables efficiency when we call simulation functions, to have it within **numpy arrays**.

Note: this is only created on a call to simulate.

In [11]:
battle.M_

### Changing AI behaviour

By default, every army group will initialise by choosing a random enemy. Once that enemy is dead, it will then randomly select a new enemy and move towards it.

This can be altered using a number of AI functions, these include:

- `random`: selects a random enemy.
- `nearest`: selects the closest enemy.
- `weakest`: selects the enemy with the lowest HP.

This can be chosen at the initial stage before the simulation starts (`init_ai`) or during the running of the simulation (`rolling_ai`).

Each element in the list corresponds to that army groups AI programming. It effects every unit within that army group.

## Running the Simulation

Simulations are run individually using the `simulate()` function. This function by default requires no parameters, but you can send some if you wish to influence some global attribute regarding the simulation.

Simulations can be called directly using the `M_` attribute and functions within the `simulator.py` file, such as `simulate_battle`.

By default, the `simulate()` function returns a `pandas.DataFrame` object which details the change of units over time. This should be stored in some object. **WARNING**: Large numbers of units within the simulation may cause this function to run slowly.

In addition, the `battle` object also stores an attribute called `sim_` which is the same as the object returned from the `simulate()` function, if you wish to refer to it later.

In [12]:
S = battle.simulate()

### The Results

The resulting object always contains *at least* the following columns:

- `army`: Which army the unit belongs to
- `allegiance`: Which team the unit belongs to
- `alive`: A boolean flag that indicates whether this unit is alive
- `x`, `y`: X-Y coordinates
- `dir_x`, `dir_y`: The direction the unit is pointing from $(0, 0)$ origin.

In [13]:
S[0]

array([( 0.05533602,  0.6957588 , 0, 20., 40.,  0.        ,  0.        , 1, 1),
       ( 0.19107129,  0.1781869 , 0, 20., 40., -0.25367546,  0.9672894 , 1, 1),
       ( 0.8749748 ,  0.05808209, 0, 20., 40., -0.78926826,  0.61404854, 1, 1),
       ( 0.71494293, -0.25887635, 0, 20., 40., -0.5684557 ,  0.8227139 , 1, 1),
       (-1.9006296 , -0.5007643 , 0, 20., 40.,  0.8530468 ,  0.52183443, 1, 1),
       (-0.6941642 , -0.8515739 , 0, 20., 40.,  0.4359336 ,  0.8999788 , 1, 1),
       (-1.0970953 ,  0.39592436, 0, 20., 40.,  0.96778107,  0.251793  , 1, 1),
       ( 0.44951084, -0.70005685, 0, 20., 40., -0.27176878,  0.9623626 , 1, 1),
       ( 1.0450847 ,  0.07905915, 0, 20., 40., -0.84872717,  0.528831  , 1, 1),
       ( 1.7567039 ,  1.1457685 , 0, 20., 40., -0.96675473, -0.25570545, 1, 1),
       ( 0.70005816,  1.3478571 , 0, 35., 45., -0.7030735 , -0.7111172 , 0, 2),
       ( 1.7654799 ,  3.1208653 , 9, 35., 45., -0.00444332, -0.99999017, 0, 2),
       ( 3.2169995 ,  0.87311226, 9, 35.

## Visualization

One of the principle aims of this project was to actively visualise the simulation in 2D space over time. To do this, we provide a number of simulation plotting functions which you can directly use with ease. The easiest way to do this, within a *Jupyter Notebook*, is to call the convenience function `sim_jupyter()`. This function uses the cached `sim_` object (i.e the last fight simulated) and draws the variables using the data stored in the `db_` object.

In [14]:
import matplotlib.pyplot as plt
# you may need to set the automatic animation.writer to ffmpeg, ensure ffmpeg is installed
plt.rcParams['animation.writer'] = 'ffmpeg'

In [15]:
battle.sim_jupyter()

### Generating HTML

The returned object from this function is from `matplotlib.animation.FuncAnimation`, an animation object. It contains a crucial function that converts this data into HTML/Javascript such that it can be displayed in Jupyter Notebook, called `to_jshtml()`.

In [16]:
html = battle.sim_jupyter(create_html=True)

In [17]:
html[:200]

'\n<link rel="stylesheet"\nhref="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css">\n<script language="javascript">\n  function isInternetExplorer() {\n    ua = navigator.userAgen'

### Displaying the HTML

IPython display then provides us with a `HTML()` function which converts HTML code text into HTML output.

Widgets at the bottom provide interaction to play, pause, rewind etc.

### Explaining the GUI

Notice that every **unit** is represented as an arrow, with the colours assigned to each **allegiance**, when a unit dies it becomes a static cross (`x`). The arrows move towards each other until in range, then attack, if they have the *aggressive* decision AI stance. The **size** of the arrow determines how powerful/important each unit type is.

Note that by default, the correct allegiance labels are passed to each group if you use `sim_jupyter` as a function from the `Battle` object. However you can also pass manual colours for each group if you don't like the defaults.

In [18]:
HTML(html)

### Extra Display Options

You may notice that random colours and labels have been assigned to **each army group**. This is because the meta-information from our `battle` object which contain labels like `[CIS, Republic]` are not passed to the drawing function. These have to be manually passed within our `quiver_fight()` function.

The function accepts a dictionary where the index is a number, referring to the army group, and the value is the text/color to display:

There's the end of this first basic introduction to **BATTLESIMULATOR**.